# Nursery_sample.py
A sample of a python script to take in data on ear counts from a maize breeding nursery and generate seed inventory and shelling (threshing) labels for seed packets.  
  
Breeding nurseries typically involving controlled pollinations that result in seed generations being different than what was planted, and in the case of crossing, the name of the material also needs to be changed to reflect the change in pedigree.  
  
We start with a nursery information file that has one entry per planted row in the field. The file has at least the following columns:  
Row (integer)  
Material (string representing the name of the variety, line, possibly a longer pedigree)  
Source ID (string representing the source of the seed used to plant the row, this permits tracking seed sources back to previous generations and inventories)  
Gen (string representing the generation of the seed that was planted)  
Poll_Type (string representing the type of pollination applied to that row)    
  
Poll_Type must be one of the following:  
SF: selfing  
SB: sibbing within the row  
CR: crossing between paired rows  
TC: topcross for paired rows  
BC: backcross for paired rows  
SF/CR: selfing some plants, crossing others  
SF/TC: selfing some plants, topcrossing others   
SF/BC: selfing some plants, backcrossing others   
FC: free-cross among arbitrary set of rows  
IN: intermate among arbitrary set of rows 

Based on the starting generation and the pollination type, the functions in the 'nursery.py' python module will create new records for the progenies with updated material and generation names. It will also produce one new record per harvested ear (if the intention is to shell ears individually). Alternatively, if bulk shelling is desired, a single record for the bulked seed will be produced. The shelling method for each row is not typically included in the nursery book, so it is specified in the script.

To determine the number of ears harvested and which rows were advanced by pollination, we need a second file containing the harvest information. The columns of this file must include at least those corresponding to:
Plot_ID: integer (to match Row in the info file)  
earno_self: integer (representing the number of selfed or sib-mated ears harvested)  
earq_self: integer (representing the quality of ears, we use 1 - 9 point scale, with 9 being best ears)  
earno_cross:integer (representing the number of crossed ears harvested, pollen came from a different row)  
earq_cross: integer (same scale as earq_self)  

Some other required information, like which rows are paired to make crosses, will be specified in the script below.  The script and module were written in Python 3.7 and were tested using pandas package version 0.25.3.

In [1]:
from pandas import Series, DataFrame 
import pandas as pd
import os as os

Set working directory to folder where ear counts and nursery information are held. Note in this case, we have a folder that starts with a number, it seems we need a double \ before that folder name or it is not recognized correctly.  
Also include the nursery.py module in this directory and read from there.

In [2]:
os.chdir("Q:\\My Drive\\TeamCorn\\FL19-20")
from nursery import * #call this AFTER defining the working directory

Input a string to use as a prefix for the current nursery in material ID's. This string will be used as a prefix for the Seed_ID labels to be generated and also will be used to name the output files. In this example, we will use "19FL", I use a standard two numeric digits for year and two characters for location.

In [3]:
nursery_pre = "19FL"

Import the ear counts and nursery information as pandas DataFrames. First we get the nursery info file. This has the information on the material, source, and generation for each row planted in the nursery.

In [4]:
info = pd.read_csv("nursery_sample_info.csv",  encoding = 'latin-1') 

Get the dimensions (row, column) of the data frame

In [5]:
info.shape

(61, 6)

look at the top and bottom of the data frame

In [6]:
info.head

<bound method NDFrame.head of     ROW             Material                  Seed ID      Gen Poll_Type  \
0     1                   A1        18FL0052-1×0109-1       S0        SF   
1     2                   A1        18FL0052-3×0103-6       S0        SF   
2     3                   A1        18FL0052-5×0102-6       S0        SF   
3     4                   A1        18FL0053-3×0102-4       S0        SF   
4     5                   A1        18FL0054-1×0096-5       F1        SF   
..  ...                  ...                      ...      ...       ...   
56   57  (B73 R-scm2*3)Ga1-S  15CL0800-BLK×15CL0799-1    BC2F1        BC   
57   58           B73 R-scm2             13CL0808-BLK   Inbred        BC   
58   59  (B73 R-scm2*3)Ga1-S  15CL0800-BLK×15CL0799-1    BC2F1        SF   
59   60  (B73 R-scm2*3)Ga1-S              16CL10008-1  BC2F1:2        SF   
60   61           B73 R-scm2             13CL0808-BLK   Inbred       OBS   

                       Notes  
0             zero ear cou

In [7]:
info.tail

<bound method NDFrame.tail of     ROW             Material                  Seed ID      Gen Poll_Type  \
0     1                   A1        18FL0052-1×0109-1       S0        SF   
1     2                   A1        18FL0052-3×0103-6       S0        SF   
2     3                   A1        18FL0052-5×0102-6       S0        SF   
3     4                   A1        18FL0053-3×0102-4       S0        SF   
4     5                   A1        18FL0054-1×0096-5       F1        SF   
..  ...                  ...                      ...      ...       ...   
56   57  (B73 R-scm2*3)Ga1-S  15CL0800-BLK×15CL0799-1    BC2F1        BC   
57   58           B73 R-scm2             13CL0808-BLK   Inbred        BC   
58   59  (B73 R-scm2*3)Ga1-S  15CL0800-BLK×15CL0799-1    BC2F1        SF   
59   60  (B73 R-scm2*3)Ga1-S              16CL10008-1  BC2F1:2        SF   
60   61           B73 R-scm2             13CL0808-BLK   Inbred       OBS   

                       Notes  
0             zero ear cou

The order of columns is not important, but the column names are critical. If needed, change the column names, as in this example.

In [8]:
info.rename(columns = {"ROW":"Row", "Seed ID":"Source ID"}, inplace = True)
keepColumns = ['Row', 'Material', 'Source ID', 'Gen', 'Poll_Type']
info = info[keepColumns]
info.head #inspect the top of the DataFrame

<bound method NDFrame.head of     Row             Material                Source ID      Gen Poll_Type
0     1                   A1        18FL0052-1×0109-1       S0        SF
1     2                   A1        18FL0052-3×0103-6       S0        SF
2     3                   A1        18FL0052-5×0102-6       S0        SF
3     4                   A1        18FL0053-3×0102-4       S0        SF
4     5                   A1        18FL0054-1×0096-5       F1        SF
..  ...                  ...                      ...      ...       ...
56   57  (B73 R-scm2*3)Ga1-S  15CL0800-BLK×15CL0799-1    BC2F1        BC
57   58           B73 R-scm2             13CL0808-BLK   Inbred        BC
58   59  (B73 R-scm2*3)Ga1-S  15CL0800-BLK×15CL0799-1    BC2F1        SF
59   60  (B73 R-scm2*3)Ga1-S              16CL10008-1  BC2F1:2        SF
60   61           B73 R-scm2             13CL0808-BLK   Inbred       OBS

[61 rows x 5 columns]>

If you need to eliminate trailing extra rows, you can use this code:

In [9]:
info = info[info.Row.notnull()]
info.tail

<bound method NDFrame.tail of     Row             Material                Source ID      Gen Poll_Type
0     1                   A1        18FL0052-1×0109-1       S0        SF
1     2                   A1        18FL0052-3×0103-6       S0        SF
2     3                   A1        18FL0052-5×0102-6       S0        SF
3     4                   A1        18FL0053-3×0102-4       S0        SF
4     5                   A1        18FL0054-1×0096-5       F1        SF
..  ...                  ...                      ...      ...       ...
56   57  (B73 R-scm2*3)Ga1-S  15CL0800-BLK×15CL0799-1    BC2F1        BC
57   58           B73 R-scm2             13CL0808-BLK   Inbred        BC
58   59  (B73 R-scm2*3)Ga1-S  15CL0800-BLK×15CL0799-1    BC2F1        SF
59   60  (B73 R-scm2*3)Ga1-S              16CL10008-1  BC2F1:2        SF
60   61           B73 R-scm2             13CL0808-BLK   Inbred       OBS

[61 rows x 5 columns]>

Make sure that values in 'Row' field are numeric

In [10]:
info['Row'] = info['Row'].astype(int)

Get the ear counts and qualities from harvest data file. The ear count files come in a variety of formats, so we need to munge them into a consistent form.

In [11]:
counts = pd.read_csv("harvest_notes_sample.csv")
counts.shape
counts.head()

,ENTITY_ID,PLOT,VALUE1,VALUE2,VALUE3,VALUE4
0,101,1,NaN,NaN,NaN,NaN
1,201,2,0.0,NaN,0.0,NaN
2,301,3,1.0,5.0,NaN,NaN
3,401,4,4.0,5.0,NaN,NaN
4,501,5,2.0,5.0,NaN,NaN


In this case we have one record for each plot with 4 harvest trait measurements named VALUE1...VALUE4. Make a new DataFrame with only the desired columns.

In [12]:
counts2 = counts[['PLOT', 'VALUE1', 'VALUE2', 'VALUE3', 'VALUE4']] 

In [13]:
counts2.shape

(61, 5)

In [14]:
counts2.head

<bound method NDFrame.head of     PLOT  VALUE1  VALUE2  VALUE3  VALUE4
0      1     NaN     NaN     NaN     NaN
1      2     0.0     NaN     0.0     NaN
2      3     1.0     5.0     NaN     NaN
3      4     4.0     5.0     NaN     NaN
4      5     2.0     5.0     NaN     NaN
..   ...     ...     ...     ...     ...
56    57     NaN     NaN     NaN     NaN
57    58     NaN     NaN     1.0     5.0
58    59     2.0     5.0     NaN     NaN
59    60     2.0     5.0     NaN     NaN
60    61     NaN     NaN     NaN     NaN

[61 rows x 5 columns]>

Reshape and/or rename the variables to make a consistent format where we have one record per plot and values for earno_self, earno_cross, earq_self, and earq_cross. The details of this depend on input file format.  
In the case of one record per row we simply have to rename the variables:

In [15]:
counts2.columns = ['Plot_ID', 'earno_self', 'earq_self', 'earno_cross',  'earq_cross']
counts2.head

<bound method NDFrame.head of     Plot_ID  earno_self  earq_self  earno_cross  earq_cross
0         1         NaN        NaN          NaN         NaN
1         2         0.0        NaN          0.0         NaN
2         3         1.0        5.0          NaN         NaN
3         4         4.0        5.0          NaN         NaN
4         5         2.0        5.0          NaN         NaN
..      ...         ...        ...          ...         ...
56       57         NaN        NaN          NaN         NaN
57       58         NaN        NaN          1.0         5.0
58       59         2.0        5.0          NaN         NaN
59       60         2.0        5.0          NaN         NaN
60       61         NaN        NaN          NaN         NaN

[61 rows x 5 columns]>

Fill in NA values with 0s:

In [16]:
counts2.fillna(0, inplace = True)

C:\Users\jholland\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


Merge the nursery information with the ear count data. Plot_ID in counts2 matches Row in info data frame. We specify this match using left_on and right_on arguments to pd.merge function.

In [17]:
info_counts = pd.merge(info, counts2, left_on='Row', right_on='Plot_ID')

note that we will have problems if Gen is missing on ANY rows, so replace with "Unknown"

In [18]:
info_counts['Gen'].fillna("Unknown", inplace=True)
info_counts['Source ID'].fillna("?", inplace=True)
info_counts['Material'].fillna("?", inplace=True)
info_counts.head

<bound method NDFrame.head of     Row             Material                Source ID      Gen Poll_Type  \
0     1                   A1        18FL0052-1×0109-1       S0        SF   
1     2                   A1        18FL0052-3×0103-6       S0        SF   
2     3                   A1        18FL0052-5×0102-6       S0        SF   
3     4                   A1        18FL0053-3×0102-4       S0        SF   
4     5                   A1        18FL0054-1×0096-5       F1        SF   
..  ...                  ...                      ...      ...       ...   
56   57  (B73 R-scm2*3)Ga1-S  15CL0800-BLK×15CL0799-1    BC2F1        BC   
57   58           B73 R-scm2             13CL0808-BLK   Inbred        BC   
58   59  (B73 R-scm2*3)Ga1-S  15CL0800-BLK×15CL0799-1    BC2F1        SF   
59   60  (B73 R-scm2*3)Ga1-S              16CL10008-1  BC2F1:2        SF   
60   61           B73 R-scm2             13CL0808-BLK   Inbred       OBS   

    Plot_ID  earno_self  earq_self  earno_cross  earq_cro

Define a list of tuples indicating the start and end rows for each block of paired rows. Paired rows are adjacent pairs of rows used for crossing. We will use this in a function to generate a dict of rows used for crossing and an arbitrary pair_code that will indicate that each row is a member of pair 0 or 1. Later in the crossing function we will check each row for the adjacent row that has the matching pair code and use that to determine the correct paired row for crossing.  
  
This list should include both regular crossing (CR) and also backcrossing (BC), but not intermating (IN) or freecrossing (FC).

In [19]:
paired_rows = [(21,24), (26,35), (47,58)]

Free crossing is between non-adjacent rows, this is handled with a dictionary of row pairs.  
This dict object free_pairs will be accessed inside the freeCrossing function to identify the correct male row 
for each female FC row that was harvested.  
free_pairs is a nested dict with this structure: {female:male} with male as a nested dict: {male:(earno, earq)}
We require that female and male keys are type int, this will fail otherwise  
  
We start with a DataFrame with columns Female, Male, Earno, and Earq and use that to create the dictionary  
You can read this information into a pandas data frame a csv file   
For this small example, we just make the DataFrame 'by hand':  

In [20]:
free_specials = pd.DataFrame({"Female":[43,44,45,45], "Male":[45,46,43,44], "earno_cross":[1,3,2,2], "earq_cross":[4,5,5,4]})

convert this data frame to a nested dict

In [21]:
free_pairs = {}
for ind in free_specials.index:
    if free_specials.loc[ind, "Female"] not in free_pairs.keys():
        free_pairs[free_specials.loc[ind, "Female"]] = {free_specials.loc[ind, "Male"]:(free_specials.loc[ind, "earno_cross"], free_specials.loc[ind, "earq_cross"])}
    else:
        free_pairs[free_specials.loc[ind, "Female"]][free_specials.loc[ind, "Male"]] = (free_specials.loc[ind, "earno_cross"], free_specials.loc[ind, "earq_cross"])

Update earno_cross fields for rows with Poll_Type FC or SF/FC so that freeCrossing() function will be triggered
#need to have earno_cross > 0. actual number of ears is recorded in free_pairs dict, we just use value of 1 to make it work.  
Only do this for female rows in the free_specials data frame, if you include a row that really does not have any ears, this may cause a failure.

In [22]:
info_counts.loc[(info_counts["Row"].isin(free_specials.Female)) & (info_counts["earno_cross"] == 0), "earno_cross"] = 1

Often some amount of munging is required to fix errors in the nursery info. These often can be done using Pandas indexing to identify the rows and columns to be adjusted. Here are some examples I've used in the past.  
  
Sometimes we miss the ear counts on a few rows and we can fix them with something like the example code below:
missing_list = range(2001,2518)
info_counts.loc[info_counts['Row'].isin(missing_list),'earno_self'] = 1

Here I have ear counts in wrong field, e.g. self count instead of cross count, and need to switch them: 
info_counts.loc[(info_counts["Row"].isin(range(1387,1681))) & (info_counts["Gen"] == "S0:1"), "earno_self"] = info_counts.loc[(info_counts["Row"].isin(range(1387,1681))) & (info_counts["Gen"] == "S0:1"), "earno_cross"] 
info_counts.loc[(info_counts["Row"].isin(range(22,203)))&(info_counts["earno_self"] == 0), "earno_self"] = info_counts.loc[(info_counts["Row"].isin(range(22,203)))&(info_counts["earno_self"] == 0), "earno_cross"]
info_counts.loc[(info_counts["Row"].isin(range(22,203)))&(info_counts["earq_self"] == 0), "earq_self"] = info_counts.loc[(info_counts["Row"].isin(range(22,203)))&(info_counts["earq_self"] == 0), "earq_cross"]
info_counts.loc[info_counts["Row"].isin(range(22,203)), ["earno_cross","earq_cross"]] = 0
    
Fix the material info for some rows
info_counts.loc[info_counts["Material"] == "Maiz Ancho Blanco C4", "Material"] = "Maiz Ancho Blanco C5"

Here we specify which rows should be shelled as single-ear, bulked, or part of a multi-row bulk. We do this by creating lists of row numbers of each group (recall that Python integer series are written as x:y+1 to encompass the range from x to y). Then we assign a shelling type to each row based on which list it is in.

In [23]:
singles_list = list(range(1, 9+1)) + list(range(21, 24+1)) + list(range(28,31+1)) + [35, 43, 45] + list(range(47,61)) 
bulks_list = list(range(10,14+1)) + list(range(15,20+1)) + [26,27,32,33,44] 

In the initial label creation, we treat multi-row bulks as regular bulks.  
Then inside the createLabels() function in the nursery module, we parse out the multi-row bulks and reformat them. For this we need the multis_list as well as multirows, a list of tuples with start/stop row numbers for each multi-row bulk

In [24]:
multis_list = list(range(37,42+1))
bulks_list = bulks_list + multis_list

In addition, we need to make a list of tuples that encode the first and last row of each group of rows that will be shelled as a multi-row bulk.

In [25]:
multirows = [(37,40), (41,42)]

Make a column called 'shell' which indicates how the row should be shelled. If one is clever, this could be specified in the nursery info file, and then we could have skipped this bit.

In [26]:
info_counts.loc[info_counts['Row'].isin(singles_list),'shell'] = 'SINGLE-EAR' 
info_counts.loc[info_counts['Row'].isin(bulks_list),'shell'] = 'BULK'
info_counts.loc[info_counts['Row'].isin(multis_list),'shell'] = 'MULTI-BULK'

Make sure info_counts starts on a plot with non-zero ear counts

In [27]:
first_nonz_self = info_counts.earno_self.to_numpy().nonzero()[0][0]
first_nonz_cross = info_counts.earno_cross.to_numpy().nonzero()[0][0]
first_nonz = min(first_nonz_self,first_nonz_cross)
info_counts = info_counts.iloc[first_nonz:,]

Call the createLabels() function in the nursery module. By default it will not return anything, but will produce output files with the shelling labels. 
The output files will appear in the working directory with the following names:  
  
nursery_pre + ' Single Ear Shelling Labels ' + date + '.csv' (single ear shelling list)  
nursery_pre + ' Bulk Shelling Labels ' + date + '.csv' (bulk shelling list)  
nursery_pre + ' Multirow Shelling Labels ' + date + '.csv' (multirow shelling list)  
  
Where nursery_pre was the string specified at the top of this script, and date is the current date taken from the operating system via time package.  
  
In this example, we pass the argument return_labels = True and assign the return object (which is a DataFrame) to sampleLabels in case you want to inspect the object.

In [28]:
sampleLabels = createLabels(inputdf = info_counts, paired_rows = paired_rows, nursery_prefix = nursery_pre, free_pairs_dict = free_pairs, multirows = multirows, multis_list = multis_list, return_labels = True) 
print("done")

Q:\My Drive\TeamCorn\FL19-20\nursery.py:638: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputdf['Row_ID'] = inputdf.apply(rowToRowID, axis = 1)
C:\Users\jholland\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\jholland\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cave

labeller for row 3
inside split_generation for row 3
plot_gen is S0
inside seed_generation
Row                            3
Material                      A1
Source ID      18FL0052-5×0102-6
Gen                           S0
Poll_Type                     SF
Plot_ID                        3
earno_self                     1
earq_self                      5
earno_cross                    0
earq_cross                     0
shell                 SINGLE-EAR
Row_ID                  19FL0003
Name: 2, dtype: object
current label =       seed_ID Material seed_gen  ear_quality          source_ID
0  19FL0003-1       A1     S0:1          5.0  18FL0052-5×0102-6
labeller for row 4
inside split_generation for row 4
plot_gen is S0
inside seed_generation
Row                            4
Material                      A1
Source ID      18FL0053-3×0102-4
Gen                           S0
Poll_Type                     SF
Plot_ID                        4
earno_self                     4
earq_self               

C:\Users\jholland\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:983: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  result = concat(values, axis=self.axis)
Q:\My Drive\TeamCorn\FL19-20\nursery.py:685: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rowset0['seed_ID'] = rowset0.loc[rowset.index[0],'seed_ID'][0:8] + '-' + str(tup[1]).zfill(4) + '-BLK'


done
